In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs
from reversible2.invert import invert

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close


In [ ]:
import sklearn.datasets
from matplotlib.patches import Rectangle
from reversible2.util import set_random_seeds
set_random_seeds(20190708, True)
X,y  = sklearn.datasets.make_moons(100, shuffle=False, noise=4e-2)
train_inputs_a = np_to_var(X[0:len(X)//2], dtype=np.float32)
train_inputs_b = np_to_var(X[len(X)//2:], dtype=np.float32)
cuda = False

plt.figure(figsize=(4,4))
plt.scatter(var_to_np(train_inputs_a)[:,0], var_to_np(train_inputs_a)[:,1], label="Class A")
plt.scatter(var_to_np(train_inputs_b)[:,0], var_to_np(train_inputs_b)[:,1], label="Class B")
ax = plt.gca()
for pt in var_to_np(th.cat((train_inputs_a, train_inputs_b))):
    rect = Rectangle(pt - 5e-2/2, 5e-2, 5e-2, facecolor='None', edgecolor='black', lw=1,
                    alpha=0.5)
    ax.add_artist(rect)
plt.legend()
plt.axis('equal')

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
from reversible2.model_and_dist import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4,
                      'weight_decay': 0.1}])


In [ ]:
from reversible2.invert import invert
n_epochs = 10001

rand_noise_factor = 5e-2

for i_epoch in range(n_epochs):
    nll_a = -th.mean(model_and_dist.get_total_log_prob(0, train_inputs_a + (th.rand_like(train_inputs_a) -0.5) * 
                                                     rand_noise_factor))
    nll_b = -th.mean(model_and_dist.get_total_log_prob(1, train_inputs_b + (th.rand_like(train_inputs_b) -0.5) * 
                                                     rand_noise_factor))
    nll = nll_a + nll_b
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        tr_out_a = model_and_dist.model(train_inputs_a)
        tr_out_b = model_and_dist.model(train_inputs_b)

        mean, std = model_and_dist.dist.get_mean_std(0)

        display_text("Epoch {:d} of {:d}\nTrain A NLL {:.1E}\nTrain B NLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs_a)).item(),
            -th.mean(model_and_dist.get_total_log_prob(1, train_inputs_b)).item(),
        ))
        fig, axes = plt.subplots(1,2, figsize=(12,5))

        for i_class in range(2):
            outs = [tr_out_a, tr_out_b][i_class]
            samples = model_and_dist.dist.get_samples(i_class,200)
            axes[1].scatter(var_to_np(samples)[:,0], var_to_np(samples)[:,1],
                            color=lighten_color(seaborn.color_palette()[i_class]),
                       s=3,
                           label="Fake "+ ["A", "B"][i_class])
            axes[1].scatter(var_to_np(outs)[:,0], var_to_np(outs)[:,1], color=seaborn.color_palette()[i_class])
            mean, std = model_and_dist.dist.get_mean_std(i_class)
            ellipse = Ellipse(var_to_np(mean), var_to_np(std[0] * 2), var_to_np(std[1] * 2),
                              edgecolor=lighten_color(seaborn.color_palette()[i_class]),
                              facecolor='None', lw=3)
            axes[1].add_artist(ellipse)
        axes[1].axis('equal')
        axes[1].set_title("Output space")




        for i_class in range(2):
            inputs = [train_inputs_a, train_inputs_b][i_class]
            examples = model_and_dist.get_examples(i_class,200)
            axes[0].scatter(var_to_np(inputs)[:,0], var_to_np(inputs)[:,1], color=seaborn.color_palette()[i_class],
                           label="Class "+ ["A", "B"][i_class])
            axes[0].scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1],
                            color=lighten_color(seaborn.color_palette()[i_class]),
                       s=3,
                           label="Fake "+ ["A", "B"][i_class])


            radians = np.linspace(0,2*np.pi,200)
            circle_points = np.stack([np.cos(radians), np.sin(radians)], axis=-1)
            circle_th = np_to_var(circle_points, device=train_inputs_a.device, dtype=np.float32)

            mean, std = model_and_dist.dist.get_mean_std(i_class)
            circle_out= mean + (circle_th * std)
            circle_in = invert(model_and_dist.model, circle_out)
            axes[0].plot(var_to_np(circle_in)[:,0], var_to_np(circle_in)[:,1],
                        alpha=1, lw=3, color=lighten_color(seaborn.color_palette()[i_class]),
                        label="Dist " + ["A", "B"][i_class])
        axes[0].axis('equal')
        axes[0].set_title("Input space")
        axes[0].legend(ncol=3)
        display_close(fig)

In [ ]:
for fig, axes in [plt.subplots(1,2, figsize=(12,5)), plt.subplots(2,1, figsize=(5,10))]:
    for i_class in range(2):
        outs = [tr_out_a, tr_out_b][i_class]
        samples = model_and_dist.dist.get_samples(i_class,200)
        axes[1].scatter(var_to_np(samples)[:,0], var_to_np(samples)[:,1],
                        color=lighten_color(seaborn.color_palette()[i_class]),
                   s=3,
                       label="Fake "+ ["A", "B"][i_class])
        axes[1].scatter(var_to_np(outs)[:,0], var_to_np(outs)[:,1], color=seaborn.color_palette()[i_class])
        mean, std = model_and_dist.dist.get_mean_std(i_class)
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0] * 2), var_to_np(std[1] * 2),
                          edgecolor=lighten_color(seaborn.color_palette()[i_class]),
                          facecolor='None', lw=3)
        axes[1].add_artist(ellipse)
    axes[1].axis('equal')
    axes[1].set_title("Output space")




    for i_class in range(2):
        inputs = [train_inputs_a, train_inputs_b][i_class]
        examples = model_and_dist.get_examples(i_class,200)
        axes[0].scatter(var_to_np(inputs)[:,0], var_to_np(inputs)[:,1], color=seaborn.color_palette()[i_class],
                       label="Class "+ ["A", "B"][i_class])
        axes[0].scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1],
                        color=lighten_color(seaborn.color_palette()[i_class]),
                   s=3,
                       label="Fake "+ ["A", "B"][i_class])


        radians = np.linspace(0,2*np.pi,200)
        circle_points = np.stack([np.cos(radians), np.sin(radians)], axis=-1)
        circle_th = np_to_var(circle_points, device=train_inputs_a.device, dtype=np.float32)

        mean, std = model_and_dist.dist.get_mean_std(i_class)
        circle_out= mean + (circle_th * std )
        circle_in = invert(model_and_dist.model, circle_out)
        axes[0].plot(var_to_np(circle_in)[:,0], var_to_np(circle_in)[:,1],
                    alpha=1, lw=3, color=lighten_color(seaborn.color_palette()[i_class]),
                    label="Dist " + ["A", "B"][i_class])
    axes[0].axis('equal')
    axes[0].set_title("Input space")
    axes[0].legend(ncol=2)
    display_close(fig)

In [ ]:
for fig, axes in [plt.subplots(1,2, figsize=(12,5)),]:
    for i_class in range(2):
        outs = [tr_out_a, tr_out_b][i_class]
        samples = model_and_dist.dist.get_samples(i_class,200)
        axes[1].scatter(var_to_np(samples)[:,0], var_to_np(samples)[:,1],
                        color=lighten_color(seaborn.color_palette()[i_class]),
                   s=3,
                       label="Fake "+ ["A", "B"][i_class])
        axes[1].scatter(var_to_np(outs)[:,0], var_to_np(outs)[:,1], color=seaborn.color_palette()[i_class])
        mean, std = model_and_dist.dist.get_mean_std(i_class)
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0] * 2), var_to_np(std[1] * 2),
                          edgecolor=lighten_color(seaborn.color_palette()[i_class]),
                          facecolor='None', lw=3)
        axes[1].add_artist(ellipse)
    axes[1].axis('equal')
    axes[1].set_title("Output space")




    for i_class in range(2):
        inputs = [train_inputs_a, train_inputs_b][i_class]
        examples = model_and_dist.get_examples(i_class,200)
        axes[0].scatter(var_to_np(inputs)[:,0], var_to_np(inputs)[:,1], color=seaborn.color_palette()[i_class],
                       label="Class "+ ["A", "B"][i_class])
        axes[0].scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1],
                        color=lighten_color(seaborn.color_palette()[i_class]),
                   s=3,
                       label="Fake "+ ["A", "B"][i_class])


        radians = np.linspace(0,2*np.pi,200)
        circle_points = np.stack([np.cos(radians), np.sin(radians)], axis=-1)
        circle_th = np_to_var(circle_points, device=train_inputs_a.device, dtype=np.float32)

        mean, std = model_and_dist.dist.get_mean_std(i_class)
        circle_out= mean + (circle_th * std)
        circle_in = invert(model_and_dist.model, circle_out)
        axes[0].plot(var_to_np(circle_in)[:,0], var_to_np(circle_in)[:,1],
                    alpha=1, lw=3, color=lighten_color(seaborn.color_palette()[i_class]),
                    label="Dist " + ["A", "B"][i_class])
    axes[0].axis('equal')
    axes[0].set_title("Input space")
    axes[0].legend(ncol=2)
    
    
    x_start = -1.2
    x_stop = 2.2
    y_start = -1
    y_stop = 1.2
    points = th.stack(th.meshgrid(th.linspace(x_start,x_stop,30),
                                  th.linspace(y_start,y_stop,30)), dim=-1)
    probs = th.exp(model_and_dist.get_total_log_prob(0,points.view(-1, points.shape[-1]))).view(
        points.shape[:2]) + th.exp(model_and_dist.get_total_log_prob(1,points.view(-1, points.shape[-1]))).view(
        points.shape[:2])
    im = axes[0].imshow(var_to_np(probs).T, origin='lower left',
                       extent=((x_start, x_stop, y_start, y_stop)), cmap=cm.Reds,
                      aspect='auto', interpolation='bilinear', vmin=0)
    plt.colorbar(im)
    display_close(fig)

In [ ]:
fig = plt.figure(figsize=(5,5))


for i_class in range(2):
    inputs = [train_inputs_a, train_inputs_b][i_class]
    examples = model_and_dist.get_examples(i_class,200)
    plt.scatter(var_to_np(inputs)[:,0], var_to_np(inputs)[:,1], color=seaborn.color_palette()[i_class],
                   label="Class "+ ["A", "B"][i_class], s=4)


plt.axis('equal')
plt.title("Input space")
plt.legend(ncol=2)


x_start = -1.2
x_stop = 2.2
y_start = -1
y_stop = 1.2
points = th.stack(th.meshgrid(th.linspace(x_start,x_stop,30),
                              th.linspace(y_start,y_stop,30)), dim=-1)
probs = (th.exp(model_and_dist.get_total_log_prob(0,points.view(-1, points.shape[-1]))).view(
    points.shape[:2]) + th.exp(model_and_dist.get_total_log_prob(1,points.view(-1, points.shape[-1]))).view(
    points.shape[:2])) / 2
im = plt.imshow(var_to_np(probs).T, origin='lower left',
                   extent=((x_start, x_stop, y_start, y_stop)), cmap=cm.Reds,
                  aspect='auto', interpolation='bilinear', vmin=0)
cbar = plt.colorbar(im)
cbar.set_label("Unconditional Likelihood")
display_close(fig)

In [ ]:
inverted.shape

In [ ]:
plt.figure(figsize=(5,5))
for i_class in range(2):
    for i_dim in range(2):
        mean, std = model_and_dist.dist.get_mean_std(i_class)
        copied = mean.clone()
        extra_vals = th.linspace((copied[i_dim] - std[i_dim] * 1.5).item(),
                                 (copied[i_dim] + std[i_dim] * 1.5).item(), 
                                 100)
        repeated = copied.repeat(100,1)
        repeated.data[:,i_dim] = extra_vals.data
        inverted = invert(model_and_dist.model, repeated)
        plt.plot(var_to_np(inverted)[:,0], var_to_np(inverted[:,1]),
                color=seaborn.color_palette()[i_class])
    
plt.axis('equal')
plt.title("Input space, directions")

In [ ]:
plt.figure(figsize=(5,5))
for i_class in range(2):
    for i_dim in range(2):
        mean, std = model_and_dist.dist.get_mean_std(i_class)
        if i_dim == 0:
            copied = mean.clone()
            other_vals = th.linspace((copied[1-i_dim] - std[1-i_dim] * 1.5).item(),
                                     (copied[1-i_dim] + std[1-i_dim] * 1.5).item(), 
                                     5)
            for o_val in other_vals:
                copied = mean.clone()
                extra_vals = th.linspace((copied[i_dim] - std[i_dim] * 1.5).item(),
                                         (copied[i_dim] + std[i_dim] * 1.5).item(), 
                                         100)
                repeated = copied.repeat(100,1)
                repeated.data[:,1-i_dim] = o_val.data
                repeated.data[:,i_dim] = extra_vals.data
                inverted = invert(model_and_dist.model, repeated)
                plt.plot(var_to_np(inverted)[:,0], var_to_np(inverted[:,1]),
                        color=seaborn.color_palette()[i_class])
        else:
            copied = mean.clone()
            extra_vals = th.linspace((copied[i_dim] - std[i_dim] * 1.5).item(),
                                     (copied[i_dim] + std[i_dim] * 1.5).item(), 
                                     100)
            repeated = copied.repeat(100,1)
            repeated.data[:,i_dim] = extra_vals.data
            inverted = invert(model_and_dist.model, repeated)
            plt.plot(var_to_np(inverted)[:,0], var_to_np(inverted[:,1]),
                    color=seaborn.color_palette()[i_class])
    
plt.axis('equal')
plt.title("Input space, directions")

In [ ]:
import sklearn.datasets
from matplotlib.patches import Rectangle
set_random_seeds(20190708, True)
X,y  = sklearn.datasets.make_moons(16, shuffle=False, noise=1e-2)
train_inputs_a = np_to_var(X[0:len(X)//2], dtype=np.float32)
train_inputs_b = np_to_var(X[len(X)//2:], dtype=np.float32)
cuda = False

plt.figure(figsize=(4,4))
plt.scatter(var_to_np(train_inputs_a)[:,0], var_to_np(train_inputs_a)[:,1], label="Class A")
plt.scatter(var_to_np(train_inputs_b)[:,0], var_to_np(train_inputs_b)[:,1], label="Class B")
ax = plt.gca()
for pt in var_to_np(th.cat((train_inputs_a, train_inputs_b))):
    rect = Rectangle(pt - 5e-2/2, 5e-2, 5e-2, facecolor='None', edgecolor='black', lw=1,
                    alpha=0.5)
    ax.add_artist(rect)
plt.legend()
plt.axis('equal')

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
from reversible2.model_and_dist import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4,
                      'weight_decay': 0.0001}])


In [ ]:
from reversible2.invert import invert
n_epochs = 10001

rand_noise_factor = 1e-2

for i_epoch in range(n_epochs):
    nll_a = -th.mean(model_and_dist.get_total_log_prob(0, train_inputs_a + (th.rand_like(train_inputs_a) -0.5) * 
                                                     rand_noise_factor))
    nll_b = -th.mean(model_and_dist.get_total_log_prob(1, train_inputs_b + (th.rand_like(train_inputs_b) -0.5) * 
                                                     rand_noise_factor))
    nll = nll_a + nll_b
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        tr_out_a = model_and_dist.model(train_inputs_a)
        tr_out_b = model_and_dist.model(train_inputs_b)

        mean, std = model_and_dist.dist.get_mean_std(0)

        display_text("Epoch {:d} of {:d}\nTrain A NLL {:.1E}\nTrain B NLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs_a)).item(),
            -th.mean(model_and_dist.get_total_log_prob(1, train_inputs_b)).item(),
        ))
        fig, axes = plt.subplots(1,2, figsize=(12,5))

        for i_class in range(2):
            outs = [tr_out_a, tr_out_b][i_class]
            samples = model_and_dist.dist.get_samples(i_class,200)
            axes[1].scatter(var_to_np(samples)[:,0], var_to_np(samples)[:,1],
                            color=lighten_color(seaborn.color_palette()[i_class]),
                       s=3,
                           label="Fake "+ ["A", "B"][i_class])
            axes[1].scatter(var_to_np(outs)[:,0], var_to_np(outs)[:,1], color=seaborn.color_palette()[i_class])
            mean, std = model_and_dist.dist.get_mean_std(i_class)
            ellipse = Ellipse(var_to_np(mean), var_to_np(std[0] * 2), var_to_np(std[1] * 2),
                              edgecolor=lighten_color(seaborn.color_palette()[i_class]),
                              facecolor='None', lw=3)
            axes[1].add_artist(ellipse)
        axes[1].axis('equal')
        axes[1].set_title("Output space")




        for i_class in range(2):
            inputs = [train_inputs_a, train_inputs_b][i_class]
            examples = model_and_dist.get_examples(i_class,200)
            axes[0].scatter(var_to_np(inputs)[:,0], var_to_np(inputs)[:,1], color=seaborn.color_palette()[i_class],
                           label="Class "+ ["A", "B"][i_class])
            axes[0].scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1],
                            color=lighten_color(seaborn.color_palette()[i_class]),
                       s=3,
                           label="Fake "+ ["A", "B"][i_class])


            radians = np.linspace(0,2*np.pi,200)
            circle_points = np.stack([np.cos(radians), np.sin(radians)], axis=-1)
            circle_th = np_to_var(circle_points, device=train_inputs_a.device, dtype=np.float32)

            mean, std = model_and_dist.dist.get_mean_std(i_class)
            circle_out= mean + (circle_th * std )
            circle_in = invert(model_and_dist.model, circle_out)
            axes[0].plot(var_to_np(circle_in)[:,0], var_to_np(circle_in)[:,1],
                        alpha=1, lw=3, color=lighten_color(seaborn.color_palette()[i_class]),
                        label="Dist " + ["A", "B"][i_class])
        axes[0].axis('equal')
        axes[0].set_title("Input space")
        axes[0].legend(ncol=3)
        display_close(fig)